In [9]:
import yaml
import pyodbc
from market_growth_analysis.etl.stagging import *
import pandas as pd

In [10]:
finantial_sheets = pd.read_csv("../../data/intermediate_02/financial-sheets.csv", index_col=0)
prices_df = pd.read_csv("../../data/raw_01/prices.csv", index_col=0)


In [11]:
# Convert 'Date' columns to datetime type
finantial_sheets['Date'] = pd.to_datetime(finantial_sheets['Date'])
prices_df['Date'] = pd.to_datetime(prices_df['Date'])

# Extract the year from 'Date' columns
finantial_sheets['Year'] = finantial_sheets['Date'].dt.year
prices_df['Year'] = prices_df['Date'].dt.year

# Perform the join based on 'Year' and 'ticker' columns
financial_and_prices = finantial_sheets.merge(prices_df, on=['Year', 'ticker'], how='inner')

In [12]:
# Drop unnecessary columns
financial_and_prices = financial_and_prices.drop(['Date_y', 'Year'], axis=1)

# Rename the 'Date_x' column if desired
financial_and_prices = financial_and_prices.rename(columns={'Date_x': 'Date'})

In [13]:
financial_and_prices['Enterprise Value (EV)'] = (financial_and_prices['Shares Outstanding'] * financial_and_prices['Close']) + financial_and_prices['Total Liabilities'] - financial_and_prices['Cash On Hand']
financial_and_prices['EV / Revenue'] = financial_and_prices['Enterprise Value (EV)'] / financial_and_prices['Revenue']
financial_and_prices['EV / EBITDA'] = financial_and_prices['Enterprise Value (EV)'] / financial_and_prices['EBITDA']
financial_and_prices['EV / EBIT'] = financial_and_prices['Enterprise Value (EV)'] / financial_and_prices['EBIT']
financial_and_prices['EV / Invested Capital'] = financial_and_prices['Enterprise Value (EV)'] / financial_and_prices['Long-Term Investments']
financial_and_prices['Free Cash Flow (FCF)'] = financial_and_prices['Cash Flow From Operating Activities'] + financial_and_prices['Cash Flow From Investing Activities']
financial_and_prices['EV / Free Cash Flow'] = financial_and_prices['Enterprise Value (EV)'] / financial_and_prices['Free Cash Flow (FCF)']
financial_and_prices['EV / FCF'] = financial_and_prices['Enterprise Value (EV)'] / financial_and_prices['Free Cash Flow (FCF)']
financial_and_prices['P/E'] = financial_and_prices['Close'] / financial_and_prices['EPS - Earnings Per Share']
financial_and_prices['P/S'] = (financial_and_prices['Shares Outstanding'] * financial_and_prices['Close']) / financial_and_prices['Revenue']
# financial_and_prices['P/B'] = financial_and_prices['Close'] / financial_and_prices['Book Value Per Share']
financial_and_prices['P/CF'] = (financial_and_prices['Shares Outstanding'] * financial_and_prices['Close']) / financial_and_prices['Cash Flow From Operating Activities']


In [14]:
financial_and_prices.reset_index(names='PK', inplace=True)

In [15]:
financial_and_prices.to_csv('../../data/processed_03/financial-prices-ratios.csv')

In [16]:
finantial_sheets.columns

Index(['Date', 'Cash On Hand', 'Receivables', 'Inventory', 'Pre-Paid Expenses',
       'Other Current Assets', 'Total Current Assets',
       'Property, Plant, And Equipment', 'Long-Term Investments',
       'Goodwill And Intangible Assets', 'Other Long-Term Assets',
       'Total Long-Term Assets', 'Total Assets', 'Total Current Liabilities',
       'Long Term Debt', 'Other Non-Current Liabilities',
       'Total Long Term Liabilities', 'Total Liabilities', 'Common Stock Net',
       'Retained Earnings (Accumulated Deficit)', 'Comprehensive Income',
       'Other Share Holders Equity', 'Share Holder Equity',
       'Total Liabilities And Share Holders Equity', 'ticker', 'industry_x',
       'sector_x', 'company_full_name_x', 'country_x', 'Net Income/Loss',
       'Total Depreciation And Amortization - Cash Flow',
       'Other Non-Cash Items', 'Total Non-Cash Items',
       'Change In Accounts Receivable', 'Change In Inventories',
       'Change In Accounts Payable', 'Change In Assets